## Data Input

**Current Data Availability:**

- Monthly excess returns for **6 anomalies**.
- **Fama–French factors** (MKT, SMB, HML).
- **Regime Classification:**
  - **Pre-Crisis:** 2003-01 to 2007-11
  - **Crisis:** 2007-12 to 2009-06
  - **Post-Crisis:** 2009-07 to 2014-05

---

## Time-Series Regression Analysis (Revised)

This model evaluates whether the risk-adjusted returns of each anomaly changed significantly during the crisis regime, while also accounting for changes in factor exposures (risk).

### Model Specifications

We estimate the following specifications with interaction terms to allow factor loadings to vary by regime.

**Model 1: Pre-crisis vs. Crisis (Baseline: Pre-crisis)**
$$R_t = \alpha_1 + \gamma_1 \cdot \text{Crisis}_t + \beta_1 \cdot \text{Factors}_t + \delta_1 (\text{Crisis}_t \times \text{Factors}_t) + \epsilon_t$$

**Model 2: Crisis vs. Post-crisis (Baseline: Crisis)**
$$R_t = \alpha_2 + \gamma_2 \cdot \text{Post}_t + \beta_2 \cdot \text{Factors}_t + \delta_2 (\text{Post}_t \times \text{Factors}_t) + \epsilon_t$$

### Variable Definitions

- **$R_t$**: The monthly long-short anomaly return.
- **$\text{Factors}_t$**: Vector of Fama–French three factors (MKT, SMB, HML).
- **Dummy Variables**:
  - **$\text{Crisis}_t$**: 1 during the crisis period (Dec 2007–June 2009), 0 otherwise.
  - **$\text{Post}_t$**: 1 during the post-crisis period (July 2009–Dec 2014), 0 otherwise.
- **Coefficients**:
  - **$\beta_1, \beta_2$**: Factor loadings during the respective **baseline** periods (Pre-crisis for Model 1; Crisis for Model 2).
  - **$\gamma_1, \gamma_2$**: Captures the abnormal returns (shift in alpha) during the Crisis and Post-crisis periods, respectively.
  - **$\delta_1, \delta_2$**: Vectors of **interaction coefficients** capturing changes in factor loadings during the Crisis and Post-crisis periods.
- **$\epsilon_t$**: The error term.


In [1]:
import pandas as pd
from IPython.display import display
import statsmodels.api as sm

In [2]:
excess_returns = pd.read_excel('./Regression Data/excess_returns.xlsx', index_col=0)
display(round(excess_returns,3))
excess_returns.info()

,Accruals,Asset Growth,BM,Gross Profit,Momentum,Leverage Ret,Regime
date,,,,,,,
2003-01-31,-0.007,0.013,-0.004,-0.013,-0.049,0.014,Pre-Crisis
2003-02-28,-0.034,0.009,-0.045,0.022,0.050,-0.006,Pre-Crisis
2003-03-31,-0.039,-0.004,-0.034,-0.012,-0.006,0.001,Pre-Crisis
2003-04-30,0.002,0.049,0.102,-0.110,-0.120,0.007,Pre-Crisis
2003-05-30,0.067,0.077,0.106,-0.171,-0.327,0.002,Pre-Crisis
...,...,...,...,...,...,...,...
2014-01-31,-0.014,0.003,-0.024,-0.144,-0.043,-0.010,Post-Crisis
2014-02-28,-0.008,-0.010,-0.018,0.004,0.010,0.017,Post-Crisis
2014-03-31,0.011,0.040,0.001,0.051,-0.001,0.000,Post-Crisis


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 137 entries, 2003-01-31 to 2014-05-30
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Accruals      137 non-null    float64
 1   Asset Growth  137 non-null    float64
 2   BM            137 non-null    float64
 3   Gross Profit  137 non-null    float64
 4   Momentum      137 non-null    float64
 5   Leverage Ret  137 non-null    float64
 6   Regime        137 non-null    object 
dtypes: float64(6), object(1)
memory usage: 8.6+ KB


In [3]:
ff_factors = pd.read_excel('./Regression Data/fama_french_factors.xlsx', index_col='date')
display(round(ff_factors.head(),3))
print(ff_factors.info())

,Unnamed: 0,Mkt-RF,SMB,HML,RF,Regime
date,,,,,,
2003-01-31,0,-0.027,0.019,0.008,0.001,Pre-Crisis
2003-02-28,1,-0.018,0.005,0.017,0.001,Pre-Crisis
2003-03-31,2,-0.004,0.005,-0.011,0.001,Pre-Crisis
2003-04-30,3,0.087,-0.000,-0.008,0.001,Pre-Crisis
2003-05-30,4,0.066,0.032,0.005,0.001,Pre-Crisis


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 137 entries, 2003-01-31 to 2014-05-30
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  137 non-null    int64  
 1   Mkt-RF      137 non-null    float64
 2   SMB         137 non-null    float64
 3   HML         137 non-null    float64
 4   RF          137 non-null    float64
 5   Regime      137 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.5+ KB
None


In [4]:
# Merge Data
# We join the anomaly returns with the Fama-French factors.
# We only need Mkt-RF, SMB, and HML from the factors df.
df_combined = pd.merge(
    excess_returns, 
    ff_factors[['Mkt-RF', 'SMB', 'HML']], 
    left_index=True, 
    right_index=True, 
    how='inner'
)
display(round(df_combined.head(),3))

,Accruals,Asset Growth,BM,Gross Profit,Momentum,Leverage Ret,Regime,Mkt-RF,SMB,HML
date,,,,,,,,,,
2003-01-31,-0.007,0.013,-0.004,-0.013,-0.049,0.014,Pre-Crisis,-0.027,0.019,0.008
2003-02-28,-0.034,0.009,-0.045,0.022,0.050,-0.006,Pre-Crisis,-0.018,0.005,0.017
2003-03-31,-0.039,-0.004,-0.034,-0.012,-0.006,0.001,Pre-Crisis,-0.004,0.005,-0.011
2003-04-30,0.002,0.049,0.102,-0.110,-0.120,0.007,Pre-Crisis,0.087,-0.000,-0.008
2003-05-30,0.067,0.077,0.106,-0.171,-0.327,0.002,Pre-Crisis,0.066,0.032,0.005


In [5]:
# List of anomaly columns (Dependent Variables)
anomalies = ['Accruals', 'Asset Growth', 'BM', 'Gross Profit', 'Momentum', 'Leverage Ret']

In [6]:
# --- Updated Regression Function with Interactions ---
def run_regressions_with_interactions(data, anomalies, dummy_col_name):
    results_list = []
    
    # Define Factor columns
    factors = ['Mkt-RF', 'SMB', 'HML']
    
    # Create Interaction Terms: Dummy * Factor
    # This captures the change in factor loading during the specific regime (delta coefficients)
    interaction_cols = []
    for f in factors:
        int_col = f"{dummy_col_name}_x_{f}"
        data[int_col] = data[dummy_col_name] * data[f]
        interaction_cols.append(int_col)
    
    # Independent Variables: Constant + Factors + Dummy + Interactions
    X_cols = factors + [dummy_col_name] + interaction_cols
    X = sm.add_constant(data[X_cols])
    
    for anomaly in anomalies:
        # Dependent Variable
        y = data[anomaly]
        
        # Fit OLS
        model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags':3})
        
        # Store results
        # We now capture the base betas (beta1/beta2) and the interaction deltas (delta1/delta2)
        results_list.append({
            'Anomaly': anomaly,
            'Alpha': model.params['const'],
            'Alpha_P_Value': model.pvalues['const'],
            
            # Regime Dummy Coefficient (Gamma)
            'Gamma_Coef': model.params[dummy_col_name],
            'Gamma_P_Value': model.pvalues[dummy_col_name],
            'Gamma_t_stat': model.tvalues[dummy_col_name],
            
            # Base Factor Loadings (Beta)
            'Base_Beta_MKT': model.params['Mkt-RF'],
            'Base_Beta_MKT_Pval': model.pvalues['Mkt-RF'],
            'Base_Beta_SMB': model.params['SMB'],
            'Base_Beta_SMB_Pval': model.pvalues['SMB'],
            'Base_Beta_HML': model.params['HML'],
            'Base_Beta_HML_Pval': model.pvalues['HML'],
            
            # Interaction Coefficients (Delta) - Change in loadings
            'Delta_MKT': model.params[f"{dummy_col_name}_x_Mkt-RF"],
            'Delta_MKT_Pval': model.pvalues[f"{dummy_col_name}_x_Mkt-RF"],
            'Delta_SMB': model.params[f"{dummy_col_name}_x_SMB"],
            'Delta_SMB_Pval': model.pvalues[f"{dummy_col_name}_x_SMB"],
            'Delta_HML': model.params[f"{dummy_col_name}_x_HML"],
            'Delta_HML_Pval': model.pvalues[f"{dummy_col_name}_x_HML"],
            
            'Adjusted R2': model.rsquared_adj,
            'Observations': int(model.nobs),
        })
        
    return pd.DataFrame(results_list)

In [7]:
# --- Run Revised Model 1: Pre-Crisis vs. Crisis ---
# Specification: R = alpha + gamma*Crisis + beta*Factors + delta(Crisis*Factors) + e

# Filter: Keep only Pre-Crisis (Baseline) and Crisis rows
df_m1 = df_combined[df_combined['Regime'].isin(['Pre-Crisis', 'Crisis'])].copy()

# Dummy: 1 if Crisis, 0 if Pre-Crisis
df_m1['CrisisDummy'] = (df_m1['Regime'] == 'Crisis').astype(int)

print("Running Model 1 (Pre-Crisis vs. Crisis) with Interactions...")
results_m1 = run_regressions_with_interactions(df_m1, anomalies, 'CrisisDummy')

results_m1.to_excel('./Regression results with interaction term/model_1_revised_results.xlsx', index=False)
display(round(results_m1, 3))

Running Model 1 (Pre-Crisis vs. Crisis) with Interactions...


,Anomaly,Alpha,Alpha_P_Value,Gamma_Coef,Gamma_P_Value,Gamma_t_stat,Base_Beta_MKT,Base_Beta_MKT_Pval,Base_Beta_SMB,Base_Beta_SMB_Pval,Base_Beta_HML,Base_Beta_HML_Pval,Delta_MKT,Delta_MKT_Pval,Delta_SMB,Delta_SMB_Pval,Delta_HML,Delta_HML_Pval,Adjusted R2,Observations
0,Accruals,0.001,0.732,-0.009,0.238,-1.179,0.079,0.460,0.227,0.217,-0.442,0.100,-0.039,0.780,-0.106,0.756,-1.222,0.042,0.284,78
1,Asset Growth,-0.008,0.031,-0.002,0.761,-0.304,0.468,0.000,0.194,0.256,0.492,0.072,-0.728,0.000,0.076,0.906,-0.002,0.996,0.214,78
2,BM,-0.003,0.329,0.022,0.054,1.924,0.896,0.000,0.480,0.024,0.657,0.005,-0.581,0.005,-0.353,0.349,0.051,0.904,0.449,78
3,Gross Profit,0.002,0.657,0.006,0.394,0.852,-0.656,0.009,-0.172,0.413,0.385,0.406,0.435,0.092,-1.451,0.004,-0.005,0.993,0.288,78
4,Momentum,0.006,0.459,-0.054,0.196,-1.294,-0.559,0.196,-0.570,0.184,0.485,0.596,-0.070,0.913,-0.459,0.748,-1.786,0.376,0.137,78
5,Leverage Ret,-0.001,0.713,0.009,0.270,1.104,-0.046,0.430,0.019,0.807,-0.056,0.547,-0.012,0.922,-0.795,0.046,0.680,0.014,0.299,78


In [8]:
# --- Run Revised Model 2: Crisis vs. Post-Crisis ---
# Specification: R = alpha + gamma*Post + beta*Factors + delta(Post*Factors) + e

# Filter: Keep only Crisis (Baseline) and Post-Crisis rows
df_m2 = df_combined[df_combined['Regime'].isin(['Crisis', 'Post-Crisis'])].copy()

# Dummy: 1 if Post-Crisis, 0 if Crisis
df_m2['PostCrisisDummy'] = (df_m2['Regime'] == 'Post-Crisis').astype(int)

print("\nRunning Model 2 (Crisis vs. Post-Crisis) with Interactions...")
results_m2 = run_regressions_with_interactions(df_m2, anomalies, 'PostCrisisDummy')

results_m2.to_excel('./Regression results with interaction term/model_2_revised_results.xlsx', index=False)
display(round(results_m2, 3))


Running Model 2 (Crisis vs. Post-Crisis) with Interactions...


,Anomaly,Alpha,Alpha_P_Value,Gamma_Coef,Gamma_P_Value,Gamma_t_stat,Base_Beta_MKT,Base_Beta_MKT_Pval,Base_Beta_SMB,Base_Beta_SMB_Pval,Base_Beta_HML,Base_Beta_HML_Pval,Delta_MKT,Delta_MKT_Pval,Delta_SMB,Delta_SMB_Pval,Delta_HML,Delta_HML_Pval,Adjusted R2,Observations
0,Accruals,-0.007,0.314,0.007,0.348,0.938,0.040,0.671,0.120,0.671,-1.664,0.001,0.137,0.231,0.012,0.971,1.562,0.002,0.351,78
1,Asset Growth,-0.010,0.107,0.009,0.191,1.308,-0.259,0.020,0.270,0.660,0.490,0.245,0.481,0.000,-0.135,0.832,0.396,0.416,0.321,78
2,BM,0.019,0.086,-0.022,0.059,-1.888,0.314,0.019,0.127,0.682,0.708,0.044,-0.187,0.228,0.561,0.204,0.494,0.282,0.324,78
3,Gross Profit,0.008,0.197,-0.007,0.391,-0.857,-0.221,0.000,-1.623,0.000,0.381,0.191,0.104,0.351,0.941,0.123,-0.876,0.029,0.238,78
4,Momentum,-0.048,0.243,0.058,0.165,1.390,-0.629,0.188,-1.030,0.449,-1.301,0.479,0.582,0.248,-0.706,0.640,-0.091,0.962,0.220,78
5,Leverage Ret,0.008,0.284,-0.008,0.302,-1.031,-0.058,0.587,-0.776,0.048,0.624,0.020,0.046,0.689,0.699,0.090,-0.495,0.084,0.244,78
